In [105]:
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from datetime import datetime
import distutils.dir_util
from keras.callbacks import CSVLogger


from keras import __version__
print(__version__)

from __future__ import division

2.0.4


In [184]:
DATA_ROOT_DIR = "/home/ubuntu/data/fish"
TEST_STG1_DIR = DATA_ROOT_DIR + '/test_stg1/'
TEST_STG2_DIR = DATA_ROOT_DIR + '/test_stg2/'
RESULTS_DIR = DATA_ROOT_DIR + '/results/'

SAMPLE_ROOT_DIR = DATA_ROOT_DIR + "/sample/"
TRAIN_SAMPLE_DIR = SAMPLE_ROOT_DIR + '/train/'
VALID_SAMPLE_DIR = SAMPLE_ROOT_DIR + '/valid/'
TRAIN_DIR = DATA_ROOT_DIR + '/train/'
VALID_DIR = DATA_ROOT_DIR + '/valid/'


In [3]:
%mkdir -p $VALID_DIR
%mkdir -p $RESULTS_DIR
%mkdir -p $TRAIN_SAMPLE_DIR
#%mkdir -p $VALID_SAMPLE_DIR # "OMITTED AS VALIDATION SIZE QUITE SMALL ANYWAY"
%mkdir -p $TEST_STG1_DIR/unknown
%mkdir -p $TEST_STG2_DIR/unknown

In [4]:
!unzip sample_submission_stg1.csv.zip
!unzip sample_submission_stg2.csv.zip
!unzip test_stg1.zip
!7za e test_stg2.7z
!unzip train.zip

unzip:  cannot find or open sample_submission_stg1.csv.zip, sample_submission_stg1.csv.zip.zip or sample_submission_stg1.csv.zip.ZIP.
unzip:  cannot find or open sample_submission_stg2.csv.zip, sample_submission_stg2.csv.zip.zip or sample_submission_stg2.csv.zip.ZIP.
unzip:  cannot find or open test_stg1.zip, test_stg1.zip.zip or test_stg1.zip.ZIP.
/bin/sh: 1: 7za: not found
unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [177]:
%cd '/home/ubuntu/data/fish/train'
!ls | while read line; do echo "$line"=`ls "$line" | wc -l`; done

/home/ubuntu/data/fish/train
ALB=1492
BET=172
DOL=108
LAG=58
NoF=406
OTHER=258
SHARK=152
YFT=630


In [6]:
print(1719+200+117+67+465+299+176+734)

3777


In [10]:
print(TRAIN_DIR)
print(TRAIN_SAMPLE_DIR)

/home/ubuntu/data/fish//train/
/home/ubuntu/data/fish//sample//train/


In [17]:

%cd /home/ubuntu/data/fish/train

import glob 
import os
import numpy as np

g = glob.glob('*')
for d in g: os.mkdir('../valid/'+d)

g = glob.glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(500): os.rename(shuf[i], '../valid/' + shuf[i])



/home/ubuntu/data/fish/train


In [180]:
%cd '/home/ubuntu/data/fish/valid'
!ls | while read line; do echo "$line"=`ls "$line" | wc -l`; done

/home/ubuntu/data/fish/valid
ALB=226
BET=28
DOL=9
LAG=9
NoF=59
OTHER=41
SHARK=24
YFT=104


In [182]:
print float(226)/float(1492)
print float(28)/float(172)
print float(9)/float(108)
print float(9)/float(58)
print float(59)/float(406)
print float(41)/float(258)
print float(24)/float(152)
print float(104)/float(630)

0.151474530831
0.162790697674
0.0833333333333
0.155172413793
0.145320197044
0.158914728682
0.157894736842
0.165079365079


In [185]:
from keras.layers import Dense, Input, Lambda, BatchNormalization, Conv2D, ZeroPadding2D, MaxPooling2D, AveragePooling2D, Activation, Flatten, Dropout
from keras.applications.resnet50 import ResNet50

resnet_base = ResNet50(include_top=False, weights='imagenet')

#classifier_input_shape = resnet_base.layers[-1].output_shape[1:] # i.e. shape of conv features (produces (None, None, None, 2048))
classifier_input_shape = (1, 1, 2048)
classifier_input = Input(shape=classifier_input_shape)

In [210]:
from keras.preprocessing.image import ImageDataGenerator
def preprocess_input(x, data_format=None):
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 4D.
        data_format: data format of the image tensor.
    # Returns
        Preprocessed tensor.
    """

    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#batch_size = 32

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/fish/train',
        target_size=(224, 224),
        batch_size=56,
        #batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        '/home/ubuntu/data/fish/valid',
        target_size=(224, 224),
        shuffle=False,
        #batch_size=batch_size,
        batch_size=20,
        class_mode='categorical')

Found 3276 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


In [211]:
def create_precomputed_data(model, batches):
    filenames = batches.filenames
    conv_features = model.predict_generator(batches, (batches.samples/batches.batch_size ))
    #conv_features = model.predict_generator(batches,
     #                              len(batches.filenames))
    labels = to_categorical(batches.classes)
    return (filenames, conv_features, labels)

In [212]:
trn_filenames, trn_conv_features, trn_labels = create_precomputed_data(resnet_base, train_generator)
val_filenames, val_conv_features, val_labels = create_precomputed_data(resnet_base, validation_generator)

In [213]:
print trn_conv_features.shape
print val_conv_features.shape

(3276, 1, 1, 2048)
(500, 1, 1, 2048)


In [214]:
assert len(trn_filenames) == 3276, "trn_filenames not as expected"
assert trn_conv_features.shape == (3276, 1, 1, 2048), "trn_conv_features not as expected"
assert trn_labels.shape == (3276, 8), "trn_labels not as expected"

assert len(val_filenames) == 500, "val_filenames not as expected"
assert val_conv_features.shape == (500, 1, 1, 2048), "val_conv_features not as expected"
assert val_labels.shape == (500, 8), "val_labels not as expected"

## save precomputed convolutional features

In [215]:
# %mkdir -p $RESULTS_DIR/ResNet50_conv_feats

import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def save_precomputed_data(filenames, conv_feats, labels, features_base_name="ResNet50_conv_feats/trn_"):
    save_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat', np.array(filenames))
    save_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat', conv_feats)
    save_array(RESULTS_DIR+"/"+features_base_name+'labels.dat', np.array(labels))
    
save_precomputed_data(trn_filenames, trn_conv_features, trn_labels, "ResNet50_conv_feats/trn_")
save_precomputed_data(val_filenames, val_conv_features, val_labels, "ResNet50_conv_feats/val_")


## load precomputed convolutional features

In [216]:
def load_array(fname):
    return bcolz.open(fname)[:]

def load_precomputed_data(features_base_name="ResNet50_conv_feats/trn_"):
    filenames = load_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat').tolist()
    conv_feats = load_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat')
    labels = load_array(RESULTS_DIR+"/"+features_base_name+'labels.dat')
    return filenames, conv_feats, labels

trn_filenames, trn_conv_features, trn_labels = load_precomputed_data("ResNet50_conv_feats/trn_")
val_filenames, val_conv_features, val_labels = load_precomputed_data("ResNet50_conv_feats/val_")

In [217]:
# Create classifier model

x= Flatten()(classifier_input)
x = Dense(8, activation='softmax')(x)
                                                     
classifier_model_v1 = Model(classifier_input, x)

#from keras.optimizers import SGD
classifier_model_v1.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])


In [218]:
def fit_precomputed_helper(model, result_dir_name, lr=0.1, nb_epoch=1):  
    K.set_value(model.optimizer.lr, lr)
    
    now = datetime.now().strftime("%Y%m%d_%H%M%S.h5")
    results_dir = RESULTS_DIR + "/" + result_dir_name + "/"
    distutils.dir_util.mkpath(results_dir)
    
    model.fit(trn_conv_features, trn_labels,
              batch_size=32, 
              epochs=nb_epoch,
              validation_data=(val_conv_features, val_labels),
              shuffle=True, 
              callbacks=[CSVLogger(results_dir+"epoch_results.csv", separator=',', append=True)])
    model.save_weights(results_dir + now)
    return model

In [225]:
classifier_model_v1 = fit_precomputed_helper(classifier_model_v1, "classifier_model_v1", lr=0.01, nb_epoch=10)

Train on 3276 samples, validate on 500 samples
Epoch 1/10
3276/3276 [==============================] - 0s - loss: 2.4111 - acc: 0.8504 - val_loss: 2.3931 - val_acc: 0.8320
Epoch 2/10
3276/3276 [==============================] - 0s - loss: 2.4110 - acc: 0.8504 - val_loss: 2.3746 - val_acc: 0.8300
Epoch 3/10
3276/3276 [==============================] - 0s - loss: 2.4110 - acc: 0.8504 - val_loss: 2.3788 - val_acc: 0.8360
Epoch 4/10
3276/3276 [==============================] - 0s - loss: 2.4110 - acc: 0.8504 - val_loss: 2.3755 - val_acc: 0.8360
Epoch 5/10
3276/3276 [==============================] - 0s - loss: 2.4110 - acc: 0.8504 - val_loss: 2.3694 - val_acc: 0.8400
Epoch 6/10
3276/3276 [==============================] - 0s - loss: 2.4110 - acc: 0.8504 - val_loss: 2.3810 - val_acc: 0.8360
Epoch 7/10
3276/3276 [==============================] - 0s - loss: 2.4110 - acc: 0.8504 - val_loss: 2.3808 - val_acc: 0.8340
Epoch 8/10
3276/3276 [==============================] - 0s - loss: 2.4110 - ac

In [226]:
classifier_model_v1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 16392     
Total params: 16,392
Trainable params: 16,392
Non-trainable params: 0
_________________________________________________________________


In [223]:
classifier_model_v1.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=200)

Epoch 1/1


ValueError: Error when checking input: expected input_7 to have shape (None, 1, 1, 2048) but got array with shape (56, 224, 224, 3)